In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
device = 'cuda'
audio_encoder_pretrain = "facebook/wav2vec2-xls-r-300m"
clip_pretrain = "openai/clip-vit-base-patch32"

In [3]:
from audio import AudioEncoder
from text import TextEncoder

audio_encoder = AudioEncoder(pretrain_name=audio_encoder_pretrain).to(device)
text_encoder = TextEncoder(pretrain_name=clip_pretrain).to(device)
# audio_encoder = AudioEncoder(pretrain_name=audio_encoder_pretrain)
# text_encoder = TextEncoder(pretrain_name=clip_pretrain)

In [4]:
from dataset import AudiocapsDataset


val = AudiocapsDataset(
    dataset_dir='data/audiocaps',
    part='val',
    audio_encoder=audio_encoder,
    text_encoder=text_encoder,
)

train = AudiocapsDataset(
    dataset_dir='data/audiocaps',
    part='train',
    audio_encoder=audio_encoder,
    text_encoder=text_encoder,
)


Audio embeddings exists.
Text embeddings exists.
Audio embeddings exists.
Text embeddings exists.


In [23]:
from typing import Any

import torch
import numpy as np
from torch.utils.data import DataLoader


def collate(items: list[dict, Any]) -> dict[str, Any]:
    audio = [item['audio'] for item in items]
    text = [item['text'] for item in items]
    path = [item['path'] for item in items]
    audio_embedding_lenght = torch.tensor([item['audio_embedding'].shape[0] for item in items])
    audio_embedding = []
    for i, item in enumerate(items):
        audio_embedding.append(torch.nn.functional.pad(torch.tensor(item['audio_embedding']), (0, 0, 0, max(audio_embedding_lenght) - audio_embedding_lenght[i])))
    audio_embedding = torch.stack(audio_embedding)

    text_embedding = torch.tensor(np.array([item['text_embedding'] for item in items]))
    return dict(
        audio=audio,
        text=text,
        path=path,
        audio_embedding=audio_embedding,
        audio_embedding_lenght=audio_embedding_lenght,
        text_embedding=text_embedding
    )


batch_size = 64

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=6, collate_fn=collate)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=6, collate_fn=collate)

In [31]:
import torch
from torch.nn.modules.loss import _Loss
from torch.nn import functional as F


class ContrastiveLoss(_Loss):
    def __init__(self, t: float = 0.1, reduction: str = "mean") -> None:
        super().__init__(reduction=reduction)
        self.t = t

    def forward(self, audio: torch.Tensor, text: torch.Tensor) -> torch.Tensor:
        audio = F.normalize(audio, p=2, dim=-1)
        text = F.normalize(text, p=2, dim=-1)
        logits = (audio @ text.T) / self.t

        target = torch.arange(audio.size(0), device=audio.device)
        loss_audio = F.cross_entropy(logits, target, reduction=self.reduction)
        loss_text = F.cross_entropy(logits.T, target, reduction=self.reduction)
        return (loss_audio + loss_text) / 2

In [33]:
example_audio = torch.randn(batch_size, 512)
example_text = torch.randn(batch_size, 512)

loss = ContrastiveLoss()
print(loss(example_audio, example_text).item())
print(loss(example_text, example_text).item())

4.270759582519531
0.0031483927741646767


In [34]:
from torch.optim import Optimizer
from torch.utils.data import Dataset
from tqdm import tqdm

from audio import AudioProjector
from metric import KNNMetric


def train_projector(
    model: AudioProjector,
    optimizer: Optimizer,
    criterion: ContrastiveLoss,
    train_loader: DataLoader,
    val_loader: DataLoader,
    metric_data: Dataset,
    metric: KNNMetric,
    epochs: int,
    device: str
) -> None:
    for epoch in range(1, epochs + 1):
        model.train()
        avg_train_loss = 0.
        avg_val_loss = 0.
        for batch in tqdm(train_loader):
            audio_embedding = batch['audio_embedding'].to(device)
            audio_embedding_lenght = batch['audio_embedding_lenght'].to(device)
            text_embedding = batch['text_embedding'].to(device)
            optimizer.zero_grad()
            audio_emb = model(audio_embedding, audio_embedding_lenght)
            loss = criterion(audio_emb, text_embedding)
            loss.backward()
            optimizer.step()
            avg_train_loss += loss.item()
        avg_train_loss /= len(train_loader)
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                audio_embedding = batch['audio_embedding'].to(device)
                audio_embedding_lenght = batch['audio_embedding_lenght'].to(device)
                text_embedding = batch['text_embedding'].to(device)
                audio_emb = model(audio_embedding, audio_embedding_lenght)
                loss = criterion(audio_emb, text_embedding)
                avg_val_loss += loss.item()
            avg_val_loss /= len(val_loader)
        
        with torch.no_grad():
            query: list[tuple[int, torch.Tensor]] = []
            key: list[tuple[int, torch.Tensor]] = []
            for item in metric_data:
                audio_embedding = torch.tensor(item['audio_embedding'], device=device).unsqueeze(0)
                audio_embedding_lenght = torch.tensor([audio_embedding.shape[1]], device=device)
                audio_emb = model(audio_embedding, audio_embedding_lenght).squeeze(0)

                audio_id = int(item['path'].split('/')[-1].split('.')[0])
                query.append(
                    (
                        audio_id,
                        audio_emb.detach().cpu()
                    )
                )
                key.append(
                    (
                        audio_id,
                        torch.tensor(item['text_embedding']).squeeze(0)
                    )
                )
            metric_value = metric(query, key)
        print(f'Epoch {epoch} train loss: {avg_train_loss:.5f}')
        print(f'Epoch {epoch} val loss: {avg_val_loss:.5f}')
        print(f'Epoch {epoch} metric: {metric_value:.5f}')

In [35]:
small_val = AudiocapsDataset(
    dataset_dir='data/audiocaps',
    part='val',
    audio_encoder=audio_encoder,
    text_encoder=text_encoder,
)

small_val.df = small_val.df[small_val.df['uniq_id'] <= 100]

metric = KNNMetric(k=4)

Audio embeddings exists.
Text embeddings exists.


In [37]:
projector = AudioProjector(in_features=512, out_features=512).to(device)
optimizer = torch.optim.Adam(projector.parameters(), lr=3e-4)
loss = ContrastiveLoss().to(device=device)

train_projector(projector, optimizer, loss, train_loader, val_loader, small_val, metric, epochs=60, device=device)

  0%|          | 0/774 [00:00<?, ?it/s]

100%|██████████| 774/774 [01:11<00:00, 10.78it/s]


Epoch 1 train loss: 3.22229
Epoch 1 val loss: 2.98363
Epoch 1 metric: 0.18069


100%|██████████| 774/774 [01:12<00:00, 10.72it/s]


Epoch 2 train loss: 2.96706
Epoch 2 val loss: 2.86507
Epoch 2 metric: 0.21040


...

Epoch 58 train loss: 2.08712
Epoch 58 val loss: 2.88628
Epoch 58 metric: 0.22525


100%|██████████| 774/774 [01:10<00:00, 10.96it/s]


Epoch 59 train loss: 2.08982
Epoch 59 val loss: 2.88453
Epoch 59 metric: 0.22277


100%|██████████| 774/774 [01:09<00:00, 11.11it/s]


Epoch 60 train loss: 2.08140
Epoch 60 val loss: 2.89051
Epoch 60 metric: 0.20792


In [3]:
from queue import PriorityQueue

import torch
from IPython.display import Audio

from dataset import AudiocapsDataset
from text import TextEncoder
from audio import AudioProjector, AudioEncoder

device = 'cuda' if torch.cuda.is_available() else 'cpu'
audio_encoder_pretrain = "facebook/wav2vec2-xls-r-300m"
clip_pretrain = "openai/clip-vit-base-patch32"


text_encoder = TextEncoder(clip_pretrain).to(device)
audio_encoder = AudioEncoder(audio_encoder_pretrain)
projector = AudioProjector(in_features=512, out_features=512).to(device)
projector.load_state_dict(torch.load('projector.pth'))

val = AudiocapsDataset(
    dataset_dir='data/audiocaps',
    part='val',
    audio_encoder=audio_encoder,
    text_encoder=text_encoder,
)

Audio embeddings exists.
Text embeddings exists.


/tmp/ipykernel_15707/2553013482.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  projector.load_state_dict(torch.load('projector.pth'))


In [6]:
from torch.nn import functional as F

def search_for_audio(text: str, data: AudiocapsDataset, top_k: int = 4):
    text_embedding = text_encoder(text)
    text_embedding = text_embedding.squeeze(0)
    text_embedding = F.normalize(text_embedding, p=2, dim=-1).detach().cpu()

    queue = PriorityQueue(maxsize=top_k)

    # Fix for many audio in data
    path_set = set()
    for item in data:
        if item['path'] in path_set:
            continue
        path_set.add(item['path'])
        audio_embedding = torch.tensor(item['audio_embedding']).to(device).unsqueeze(0)
        audio_embedding_lenght = torch.tensor([audio_embedding.shape[1]]).to(device)
        with torch.no_grad():
            audio_emb = projector(audio_embedding, audio_embedding_lenght)
        audio_emb = F.normalize(audio_emb, p=2, dim=-1).squeeze(0).detach().cpu()
        score = torch.dot(audio_emb, text_embedding).item()
        if queue.full():
            if score > queue.queue[0][0]:
                queue.get()
                queue.put((score, item['path']))
        else:
            queue.put((score, item['path']))

    print(f'For text "{text}" closest audio:')
    for _ in range(queue.qsize()):
        score, path = queue.get()
        # Show audio
        print(f"Simularity score: {score:.4f}")
        display(Audio(path, autoplay=True))


search_for_audio("Bird", val, top_k=3)

For text "Bird" closest audio:
Simularity score: 0.2634


Simularity score: 0.2688


Simularity score: 0.2743
